<center><h1>Least Squares Support Vector Classifier</h1></center>

## Summary:
1. [Introduction](#introduction)


2. [Using the classifier](#using_classifier)

    2.1 [Different Kernels](#cpu_version)
    

# 1. Introduction <a class="anchor" id="introduction"></a>

The Least Squares Support Vector Machine (LSSVM) is a variation of the original Support Vector Machine (SVM) in which we have a slight change in the objective and restriction functions that results in a big simplification of the optimization problem.

First, let's see the optimization problem of an SVM:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + C \sum_{i=1}^{n} \xi_i &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b)\geq 1 - \xi_i, && i = 1,..., n \\
         && \xi_i \geq 0,                            && i = 1,..., n
\end{align}
$$

In this case, we have a set of inequality restrictions and when solving the optimization problem by it's dual we find a discriminative function, adding the kernel trick, of the type:


$$ f(\vec{x}) = sign \ \Big( \sum_{i=1}^{n} \alpha_i^o y_i K(\vec{x}_i,\vec{x}) + b_o \Big) $$

Where $\alpha_i^o$ and $b_o$ denote optimum values. Giving enough regularization (smaller values of $C$) we get a lot of $\alpha_i^o$ nulls, resulting in a sparse model in which we only need to save the pairs $(\vec{x}_i,y_i)$ which have the optimum dual variable not null. The vectors $\vec{x}_i$ with not null $\alpha_i^o$ are known as support vectors (SV).



In the LSSVM case, we change the inequality restrictions to equality restrictions. As the $\xi_i$ may be negative we square its values in the objective function:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + \gamma \frac{1}{2}\sum_{i=1}^{n} \xi_i^2 &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b) = 1 - \xi_i, && i = 1,..., n
\end{align}
$$


The dual of this optimization problem results in a system of linear equations, a set of Karush-Khun-Tucker (KKT) equations:

$$
\begin{bmatrix} 
    0 & \vec{d}^T \\
    \vec{y} & \Omega + \gamma^{-1} I 
\end{bmatrix}
\
\begin{bmatrix} 
    b  \\
    \vec{\alpha}
\end{bmatrix}
=
\begin{bmatrix} 
    0 \\
    \vec{1}
\end{bmatrix}
$$

Where, with the kernel trick, &nbsp; $\Omega_{i,j} = y_i y_j K(\vec{x}_i,\vec{x}_j)$,  &nbsp;  $\vec{y} = [y_1 \ y_2 \ ... \ y_n]^T$, &nbsp; $\vec{\alpha} = [\alpha_1 \ \alpha_2 \ ... \ \alpha_n]^T$ &nbsp;  e &nbsp; $\vec{1} = [1 \ 1 \ ... \ 1]^T$.

The discriminative function of the LSSVM has the same form of the SVM but the $\alpha_i^o$ aren't usually null, resulting in a bigger model. The big advantage of the LSSVM is in finding it's parameters, which is reduced to solving the linear system of the type:

$$ A\vec{z} = \vec{b} $$

A well-known solution of the linear system is when we minimize the square of the residues, that can be written as the optimization problem:

$$
\begin{align}
    minimize && f_o(\vec{z})=\frac{1}{2}||A\vec{z} - \vec{b}||^2\\
\end{align}
$$

And have the analytical solution:

$$ \vec{z} = A^{\dagger} \vec{b} $$

Where $A^{\dagger}$ is the pseudo-inverse defined as:

$$ A^{\dagger} = (A^T A)^{-1} A^T$$

# 2. Using the classifier <a class="anchor" id="using_classifier"></a>

In [85]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler

from lssvm import LSSVC
from utils.encoding import dummie2multilabel

In [86]:
import pandas as pd
data = pd.read_csv('./langfeatures.csv', sep=",")
print(data)

import numpy as np
import pandas as pd

num_col = len(data.columns)
num_row = len(data.index)

normdata = data.iloc[:,0:num_col-1]

# remove min, divide by variance
for i in range(num_col-2):
    x = data.iloc[:,i]
    normdata.iloc[:,i] = (x-x.min())/(x.max()-x.min())
#print(normdata)



     0.509438652  0.490447973  0.459416789  0.389270137  0.389622357  \
0       0.511510     0.424546     0.388545     0.330314     0.317644   
1       0.539133     0.463781     0.433143     0.367356     0.376333   
2       0.513139     0.409610     0.436496     0.372693     0.319311   
3       0.463976     0.435471     0.377037     0.360475     0.311384   
4       0.476905     0.495385     0.394527     0.361846     0.340298   
..           ...          ...          ...          ...          ...   
647     0.448289     0.430851     0.372547     0.330591     0.289256   
648     0.273047     0.226982     0.200011     0.198919     0.181098   
649     0.587593     0.517364     0.430658     0.420942     0.383764   
650     0.433647     0.379705     0.352394     0.298677     0.280193   
651     0.403983     0.285895     0.311456     0.242800     0.212219   

     0.364762973  0.288698906  0.302846864  0.266432168  0.253494728  ...  \
0       0.316382     0.266822     0.263619     0.221372   

In [87]:
import numpy.linalg as linalg

data_var = data.iloc[:,0:len(data.columns)-1]
data_meanresidual = data_var-np.mean(data_var,axis=0)/np.std(data_var,axis=0)

cov = np.cov(normdata, rowvar=False)

eig_val, eig_vect = linalg.eig(cov)

def checkSorted(arr):
    n=len(arr)

    if(n<2):
        return True

    flag = 1
    for i in range(n-1):
        if(arr[i]<=arr[i+1]):
            flag=0
    return flag

if checkSorted(eig_val):
    print("Sorted")
else:
    print("Eigval Not Sorted")

id_sorted = np.argsort(eig_val)[::-1]

eigval_sorted = eig_val[id_sorted]
eigvect_sorted = eig_vect[:,id_sorted]

if checkSorted(eigval_sorted):
    print("resort Sorted")
else:
    print("resort Eigval Not Sorted")

print(len(eig_val))



Eigval Not Sorted
resort Sorted
24


In [88]:
n=6

part_eigval = eigval_sorted[0:n]
part_eigvect = eigvect_sorted[:,0:n]

sqparteigval = part_eigval*part_eigval.transpose()
sqeigval = eigval_sorted*eigval_sorted.transpose()

p = np.sqrt(sqparteigval.sum())
q = np.sqrt(sqeigval.sum())

pca = p/q
print(n)
print(pca)
    
data_reduced = np.dot(part_eigvect.transpose(), 
normdata.transpose()).transpose()

print(normdata.shape)
print(data_meanresidual.shape)

6
0.9998483241201501
(652, 24)
(652, 24)


In [89]:
def shuffle_slice(ls, indexes):
     ls_ = []
     for i in indexes:
        ls_.append(ls[i])
     return ls

In [90]:
import math
frac = 60
length = len(data_reduced)
limit = math.floor(frac*length/100)

Xtemp = data_reduced
ytemp = data.iloc[:,len(data.columns)-1]

X = np.array(Xtemp)
y = np.array(ytemp)


In [91]:
# Preprocessing

# Import digits recognition dataset (from sklearn)
# X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2020)

# Scaling features (from sklearn)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

X_train.shape: (391, 6)
X_test.shape:  (261, 6)
y_train.shape: (391,)
y_test.shape:  (261,)
np.unique(y_train): [1 2 3 4 5 6 7]
np.unique(y_test):  [1 2 3 4 5 6 7]


## Different Kernels <a class="anchor" id="cpu_version"></a>

In [92]:
# Use the classifier with different kernels

print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
print('acc_test = ', acc, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')

Gaussian kernel:
acc_test =  0.8467432950191571 

Polynomial kernel:
acc_test =  0.8544061302681992 

Linear kernel:
acc_test =  0.8735632183908046 



In [93]:
yprederr = lssvc.predicterror(X_ts_norm)
print(yprederr)

AttributeError: 'LSSVC' object has no attribute 'predicterror'

In [ ]:
lssvc.dump('model')
loaded_model = LSSVC.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(loaded_model.predict(X_ts_norm))
    )
)

acc_test =  0.8735632183908046
acc_test =  0.8735632183908046
